In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Run this cell once, then go to 'Runtime' -> 'Restart Runtime', then comment this cell out
!pip install miscnn
!pip install matplotlib==3.3.1

In [14]:
# Standard libraries
import tensorflow as tf
import os
import random
import numpy as np
import argparse
import math
import json
import skimage
import pandas as pd

import matplotlib.pyplot as plt
from skimage.transform import resize

# MIScnn
from miscnn.neural_network.metrics import identify_axis
from miscnn.processing.data_augmentation import Data_Augmentation
from miscnn.processing.subfunctions.normalization import Normalization
from miscnn.processing.subfunctions.resize import Resize
from miscnn.processing.subfunctions.clipping import Clipping
from miscnn.processing.subfunctions.resampling import Resampling
from miscnn.processing.preprocessor import Preprocessor
from miscnn.neural_network.model import Neural_Network
from miscnn.neural_network.metrics import *
import miscnn.neural_network.metrics

from miscnn.neural_network.architecture.unet.standard import Architecture

from miscnn.evaluation.cross_validation import run_fold, load_disk2fold
from miscnn.data_loading.interfaces.image_io import Image_interface
from miscnn.data_loading.data_io import Data_IO
from miscnn.data_loading.data_io import backup_evaluation

# Tensorflow
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler

In [ ]:
#Generating interface: single channel with 2 classes
interface = Image_interface(pattern = "img[0-9]*" # Pattern may differ depending on folder structure
                                 , img_type = 'rgb'
                                 , img_format = 'png'
                                 , classes = 2)

# path to data folder
data_path = "/path/to/dataset" # Path to DRIVE dataset

# Generating dataloader
data_io = Data_IO(interface, data_path, delete_batchDir=False)

# Sample list
sample_list = data_io.get_indiceslist()
sample_list.sort()

# Basic checks
print("All samples: " + str(len(sample_list)))
sample = data_io.sample_loader(sample_list[0], load_seg=True)  
print("Image dimension check:",sample.img_data.shape, sample.seg_data.shape)

In [8]:
# Data augmentation
data_aug = Data_Augmentation(cycles=6, scaling=False, rotations=True, elastic_deform=False, mirror=True,
                             brightness=True, contrast=True, gamma=False, gaussian_noise=False)


# Data preprocessing parameters
sf_normalize = Normalization(mode='z-score')
sf_resize = Resize((512,512))
subfunctions = [sf_resize, sf_normalize]


# Create preprocessing class
pp = Preprocessor(data_io
                  , data_aug=data_aug
                  , batch_size=2
                  , prepare_subfunctions=True
                  , subfunctions=subfunctions
                  , prepare_batches=False
                  , analysis="fullimage"
                  , use_multiprocessing=False)

In [ ]:
# U-Net
architecture = Architecture()

# Here the loss function is chosen:
loss = asym_unified_focal_loss()

# Create the Neural Network model
model = Neural_Network(architecture=architecture
                      , preprocessor=pp
                      , loss=loss
                      , metrics=[dice_soft]
                      , batch_queue_size=3
                      , workers=1
                      , learning_rate=1e-3)

In [10]:
# Learning rate scheduler
cb_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, mode='min', min_delta=1e-7, cooldown=1, min_lr=1e-7)

cb_es = EarlyStopping(monitor='val_loss', min_delta=1e-7, patience=20, verbose=1, mode='min')

In [ ]:
# Run pipeline for cross-validation fold
run_fold(fold=0
         , model=model
         , epochs=300
         , evaluation_path='/path/to/folder'
         , draw_figures=True
         , callbacks=[cb_lr,cb_es]
         , save_models=True
        )

**Evaluation code**

In [ ]:
# This cell is used for evaluation

path = os.path.join('/path/to/folder','fold_0','model.hdf5')
training,validation = load_disk2fold(os.path.join('/path/to/folder','fold_0','testing.json'))
evaluation_path = '/path/to/folder/fold_0'

# load model
model.load(path)


def load_disk2fold(file_path):
    with open(file_path, "r") as jsonfile:
        sampling = json.load(jsonfile)
        if "TRAINING" in sampling : training = sampling["TRAINING"]
        else : training = None
        if "VALIDATION" in sampling : validation = sampling["VALIDATION"]
        else : validation = None
    return training, validation


# Calculate class-wise dice similarity coefficient
def compute_dice(truth, pred, classes):
    dice_scores = []
    # Compute Dice for each class
    i = 1
    try:
        pd = np.equal(pred, i)
        gt = np.equal(truth[:,:,0], i)
        dice = 2*np.logical_and(pd, gt).sum()/(pd.sum() + gt.sum())
        dice_scores.append(dice)
    except ZeroDivisionError:
        dice_scores.append(0.0)
    # Return computed Dice scores
    return dice_scores


def compute_rest(truth, pred, classes):
    precision_scores = []
    recall_scores = []
    # Compute precision, recall scores for each class
    i = 1
    try:
        pd = np.equal(pred, i)
        gt = np.equal(truth[:,:,0], i)
        tp = np.logical_and(pd,gt).sum()
        fp = np.logical_and(pd,np.logical_not(gt)).sum()
        fn = np.logical_and(gt,np.logical_not(pd)).sum()
        precision = tp/(tp+fp)
        precision_scores.append(precision)
    except ZeroDivisionError:
        precision_scores.append(0.0) 

    try:
        recall = tp/(tp+fn)
        recall_scores.append(recall)
    except ZeroDivisionError:
        recall_scores.append(0.0)
            
    # Return computed precision, recall scores 
    return precision_scores, recall_scores

# Initialize detailed validation scoring file
classes_1 = ["dice_score"]
classes_2 = ["precision_score"]  
classes_3 = ["recall_score"]          
header = ["sample_id"]
header.extend(classes_1)
header.extend(classes_2)
header.extend(classes_3)
backup_evaluation(header, evaluation_path, start=True)

for sample_index in validation:
        pred = model.predict([sample_index],return_output=True,activation_output=True)

        # get prediction from list of predictions
        pred = pred[0]

        # Load the sample
        sample = model.preprocessor.data_io.sample_loader(sample_index,
                                                          load_seg=True,
                                                          load_pred=False)
        # Access image and ground truth segmentation data
        img, seg = sample.img_data, sample.seg_data

        # resize segmentation to size of prediction activations
        seg_act = resize(seg,(pred.shape[0], pred.shape[1]),order=0, preserve_range=True,anti_aliasing=False)

        # convert softmax predictions to classes
        pred = np.argmax(pred,axis=-1)
        
        # resize final prediction to original image shape
        pred = resize(pred,(seg.shape[0], seg.shape[1]),order=0, preserve_range=True,anti_aliasing=False)

        # Compute segmentation metrics
        dice_scores = compute_dice(seg, pred, 1)
        precision_scores, recall_scores = compute_rest(seg, pred, 1)
        
        # Save detailed validation scores to file
        scores = [sample_index]
        scores.extend(dice_scores)
        scores.extend(precision_scores)
        scores.extend(recall_scores)
        backup_evaluation(scores, evaluation_path, start=False)   

In [ ]:
# Get final scores 
results_csv = pd.read_csv('/path/to/folder/fold_0/results.tsv', sep='\t')
print('Dice score in holdout test set: ', results_csv['dice_score'].mean())